# utils & framework

In [1]:
from collections.abc import Iterable

from IPython.core.display import Markdown
from sympy import *

tag_id = 0
var_id = 0

conditions = []
values = {}


def save_as_condition(condition):
    global tag_id, conditions
    cached_condition_id = cond_id
    conditions.append(condition)
    cond_id += 1
    return cached_condition_id


def substitute_values(expression):
    while True:
        left = False
        for (key, value) in values.items():
            before = str(expression)
            expression = expression.subs(key, value)
            expression = simplify(expression)
            if str(expression) != before:
                left = True

        if not left:
            break

    return expression


def save_as_variable(symbol, value):
    global values
    if symbol in values.keys():
        raise ValueError(f"variable {symbol} already exists")
    values[symbol] = substitute_values(value)


def get_conditions(ids) -> tuple:
    return tuple(map(lambda i: conditions[i], ids))


def beautiful_print(expression, message="", tag=None):
    # escape the message for LaTeX and append the expression
    inline = not message.endswith('$')
    tag_str = f'\\tag{{{tag}}}' if tag is not None else ""
    if inline:
        full_latex = message + '$' + latex(expression) + '$'
    else:
        full_latex = message + '$' + latex(expression) + tag_str + '$$'
    display(Markdown(full_latex))
def beautiful_peek(symbol, message="$"):
    value = substitute_values(symbol)
    beautiful_print(Eq(symbol, value, evaluate=False), message)
    save_as_variable(symbol, value)

def beautiful_solve(cond, variables):
    cond = [cond] if not isinstance(cond, Iterable) else cond
    variables = [variables] if not isinstance(variables, Iterable) else variables
    solution = solve(get_conditions(cond), tuple(variables), dict=True)[0]
    conclusions = []
    display(Markdown(f"${','.join(f"\\left ( {i} \\right ) " for i in cond)} \\Longrightarrow$"))
    for (variable, conclusion) in solution.items():
        beautiful_print(Eq(variable, conclusion), "$")
        save_as_variable(variable, conclusion)
        conclusions.append(conclusion)
    return tuple(conclusions)


def beautiful_condition(expression, message="$"):
    beautiful_print(expression(False) if callable(expression) else expression, message, tag_id)
    return save_as_condition(expression(True) if callable(expression) else expression)


def beautiful_compute(symbol, expression=None, message="$", tag: bool = False):
    beautiful_print(Eq(symbol, expression(False) if callable(expression) else expression), message,
                    tag_id if tag else None)
    save_as_variable(symbol, expression)




# Define variables

In [2]:
v = symbols('v')
p = symbols('p')
r = symbols('r')

pitch = symbols('pitch')
t = symbols('t')
varphi = Function('varphi')(t)
varphi_d1 = diff(varphi, t, 1)
varphi_d2 = diff(varphi, t, 2)

rho = Function('rho')(t)
rho_d1 = diff(rho, t, 1)
rho_d2 = diff(rho, t, 2)
h = Function('h')(t)
h_d1 = diff(h, t, 1)
h_d2 = diff(h, t, 2)



In [3]:
cond1 = beautiful_condition(lambda e: Eq(tan(pitch), p / (2 * pi * r)), "by geometry:$")
cond2 = beautiful_condition(lambda e: Eq(sin(pitch), h_d1 / v), "by geometry:$")
cond3 = beautiful_condition(lambda e: Eq(cos(pitch), varphi_d1 * r / v), "by geometry:$")
_ = beautiful_solve([cond1, cond2, cond3], [pitch, h_d1, varphi_d1])



by geometry:$$\tan{\left(pitch \right)} = \frac{p}{2 \pi r}\tag{0}$$

UnboundLocalError: cannot access local variable 'cond_id' where it is not associated with a value

In [4]:
beautiful_condition(Eq(diff(v,t,evaluate= false),0),"by const vel:$")
beautiful_peek(h_d2,"so as to:$")
beautiful_peek(varphi_d2,"so as to:$")


by const vel:$$\frac{d}{d t} v = 0\tag{3}$$

so as to:$$\frac{d^{2}}{d t^{2}} h{\left(t \right)} = \frac{\partial}{\partial t} \frac{p v}{r \sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}$$

so as to:$$\frac{d^{2}}{d t^{2}} \varphi{\left(t \right)} = \frac{\partial}{\partial t} \frac{2 \pi v}{r \sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}$$

In [5]:
beautiful_compute(rho,r,"by const radius:$")
beautiful_peek(rho_d1,"so as to:$")
beautiful_peek(rho_d2,"so as to:$")


by const radius:$$\rho{\left(t \right)} = r$$

so as to:$$\frac{d}{d t} \rho{\left(t \right)} = \frac{d}{d t} r$$

so as to:$$\frac{d^{2}}{d t^{2}} \rho{\left(t \right)} = \frac{d^{2}}{d t^{2}} r$$

In [6]:
velocity = Matrix([rho_d1,rho *varphi_d1,h_d1])
beautiful_print(velocity, r"velocity vector in polar coordinates:$")
velocity = substitute_values(velocity)
beautiful_print(velocity, r"and inlined:$")


velocity vector in polar coordinates:$$\left[\begin{matrix}\frac{d}{d t} \rho{\left(t \right)}\\\rho{\left(t \right)} \frac{d}{d t} \varphi{\left(t \right)}\\\frac{d}{d t} h{\left(t \right)}\end{matrix}\right]$$

and inlined:$$\left[\begin{matrix}\frac{d}{d t} r\\\frac{2 \pi v}{\sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}\\\frac{p v}{r \sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}\end{matrix}\right]$$

In [7]:
acceleration = Matrix([rho_d2- rho* varphi_d1**2 ,  2*varphi_d1*rho_d1, h_d2])
beautiful_print(acceleration, r"acceleration vector in polar coordinates:$")
acceleration = substitute_values(acceleration)
beautiful_print(acceleration, r"and inlined:$")


acceleration vector in polar coordinates:$$\left[\begin{matrix}- \rho{\left(t \right)} \left(\frac{d}{d t} \varphi{\left(t \right)}\right)^{2} + \frac{d^{2}}{d t^{2}} \rho{\left(t \right)}\\2 \frac{d}{d t} \rho{\left(t \right)} \frac{d}{d t} \varphi{\left(t \right)}\\\frac{d^{2}}{d t^{2}} h{\left(t \right)}\end{matrix}\right]$$

and inlined:$$\left[\begin{matrix}\frac{d^{2}}{d t^{2}} r - \frac{4 \pi^{2} v^{2}}{r \left(\frac{p^{2}}{r^{2}} + 4 \pi^{2}\right)}\\\frac{4 \pi v \frac{d}{d t} r}{r \sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}\\\frac{\partial}{\partial t} \frac{p v}{r \sqrt{\frac{p^{2}}{r^{2}} + 4 \pi^{2}}}\end{matrix}\right]$$

In [8]:
beautiful_compute(v,S(20),"assign:$")
beautiful_compute(p,S(0.2),)
beautiful_compute(r,S(0.5),)

assign:$$v = 20$$

$$p = 0.2$$

$$r = 0.5$$

In [9]:
velocity = substitute_values(velocity)
beautiful_print(velocity, r"and evaluated vel:$")
acceleration = substitute_values(acceleration)
beautiful_print(acceleration, r"and evaluated acc:$")


and evaluated vel:$$\left[\begin{matrix}\frac{d}{d t} 0.5\\\frac{40 \pi}{\sqrt{0.16 + 4 \pi^{2}}}\\\frac{8.0}{\sqrt{0.16 + 4 \pi^{2}}}\end{matrix}\right]$$

and evaluated acc:$$\left[\begin{matrix}\frac{d^{2}}{d t^{2}} 0.5 - \frac{3200.0 \pi^{2}}{0.16 + 4 \pi^{2}}\\\frac{160.0 \pi \frac{d}{d t} 0.5}{\sqrt{0.16 + 4 \pi^{2}}}\\\frac{d}{d t} \frac{8.0}{\sqrt{0.16 + 4 \pi^{2}}}\end{matrix}\right]$$